In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from itertools import chain

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K


In [6]:
img_width = 128
img_height = 128
img_channels = 3

train_path = 'U_NET/train/'
test_path = 'U_NET/validation/'

In [7]:
train_ids = next(os.walk(train_path))[1]
test_ids = next(os.walk(test_path))[1]

In [8]:
X_train = np.zeros((len(train_ids), 128, 128, 3), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), 128, 128, 1), dtype=np.bool)

In [9]:

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    
    path = train_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    img = resize(img, (128, 128), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((128, 128, 1), dtype=np.bool)
    
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (128, 128), mode='constant', preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
        Y_train[n] = mask 

100%|██████████| 670/670 [06:41<00:00,  1.67it/s]


In [10]:
X_test = np.zeros((len(test_ids), 128, 128, 3), dtype=np.uint8)
sizes_test = []

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = test_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (128, 128), mode='constant', preserve_range=True)
    X_test[n] = img

100%|██████████| 65/65 [00:02<00:00, 27.13it/s]


In [11]:
def iou_loss(y_true, y_pred):
    
    intersection = K.sum(K.abs(y_true + y_pred), axis = -1)
    union = K.sum((y_true, -1) + K.sum(y_pred, -1) - intersection)
    
    return (intersection +1) / (union + 1)